In [1]:
import random
import pandas as pd
import numpy as np
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import KFold
import sys
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

In [2]:
true_class, false_class = 'T','F'
# random.seed(1245)

#  Reading the data and converting into DataFrame

In [3]:
read_file = lambda x : [tuple(line.strip().split("\t")) for line in open(x).readlines() if len(line.strip()) > 0]
assign_class = lambda x,y : [l + (y, ) for l in x]

true_lines = assign_class(read_file("./hotelT-train.txt"), true_class)
false_lines = assign_class(read_file("./hotelF-train.txt"), false_class)

all_lines = true_lines + false_lines

random.shuffle(all_lines)

raw_data_pd = pd.DataFrame(all_lines, columns = ["id", "sentence", "class"])

# Tokenizing the sentences, removing stop words

In [4]:
stop_words = set(stopwords.words('english'))
raw_data_pd["sentence_tokens"] = raw_data_pd["sentence"].apply(lambda x : [token.lower() for token in word_tokenize(x) if token.isalpha() and token.lower() not in stop_words])

# Extracting features from reviews
    - Using all the words as features k-fold accuracy ~ 48 %
    - Using all the words with count > 1 as features k-fold accuracy ~ 47 % (works slightly worst)

In [5]:
vocabulary = nltk.FreqDist()
for tokens in raw_data_pd["sentence_tokens"].tolist():
    for token in tokens:
        vocabulary[token] += 1

In [15]:
def review_features(tokens):
#     return {token: True for token in tokens if vocabulary[token] > 1}
    return {token: True for token in tokens}

raw_data = raw_data_pd[["sentence_tokens", "class"]].apply(lambda x : (review_features(x["sentence_tokens"]), x["class"]), axis = 1)

# K-Fold cross validation

In [20]:
kf = KFold(n_splits=10)
accuracy = []
random.shuffle(raw_data)
for train_idx, test_idx in kf.split(raw_data):
    train_set, test_set = raw_data[train_idx].tolist(), raw_data[test_idx].tolist()
    classifier = nltk.NaiveBayesClassifier.train(train_set)
    accuracy.append(nltk.classify.accuracy(classifier, test_set))
print np.average(accuracy)

0.481672203765
